기본 세팅

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!pip install konlpy
!pip install kss

depression score model

In [ ]:
import pandas as pd
import numpy as np
import kss
import re
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import sequence
import math

def convert_to_ord(data):
    try:
        return [ord(xx) for xx in data]
    except:
        print(data)

df_test = pd.read_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/test.csv')
df_after = []
model_yn = load_model('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/model_yn.h5')
model_09 = load_model('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/model_09.h5')

for i in range(len(df_test)):
    sentence_list = kss.split_sentences(df_test.iloc[i,0])
    for j in range(len(sentence_list)):
        df_after.append({'sentence':' '.join(re.sub(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]', ' ', str(sentence_list[j]).strip()).split()),'ord':np.nan,'label_1':np.nan,'label_2':np.nan})

df_after = pd.DataFrame(df_after)

df_after['ord'] = df_after['sentence'].map(convert_to_ord)

max_len = 64

sentence_ord = sequence.pad_sequences(df_after['ord'], maxlen = max_len)
sentence_ord = np.array(sentence_ord).reshape((len(sentence_ord), max_len))

df_after['label_1'] = model_yn.predict_classes(sentence_ord)
df_after['label_2'] = model_09.predict_classes(sentence_ord)

df_after['label_2'][df_after['label_1']==0] = np.nan

df_yn = df_after.groupby(['label_1']).count()
df_09 = df_after.groupby(['label_1','label_2']).count()

Y = len(df_after[df_after['label_1']==1])
N = len(df_after[df_after['label_1']==0])
all = Y+N
p = Y/all

phq_0 = len(df_after[df_after['label_2']==0])
phq_1 = len(df_after[df_after['label_2']==1])
phq_2 = len(df_after[df_after['label_2']==2])
phq_3 = len(df_after[df_after['label_2']==3])
phq_4 = len(df_after[df_after['label_2']==4])
phq_5 = len(df_after[df_after['label_2']==5])
phq_6 = len(df_after[df_after['label_2']==6])
phq_7 = len(df_after[df_after['label_2']==7])
phq_8 = len(df_after[df_after['label_2']==8])
phq_9 = len(df_after[df_after['label_2']==9])

score_1 = 3*math.tanh(1.477*phq_1/all)
score_2 = 3*math.tanh(0*phq_2/all)
score_3 = 3*math.tanh(0.7032*phq_3/all)
score_4 = 3*math.tanh(0.1584*phq_4/all)
score_5 = 3*math.tanh(0.6516*phq_5/all)
score_6 = 3*math.tanh(0.7536*phq_6/all)
score_7 = 3*math.tanh(0.2094*phq_7/all)
score_8 = 3*math.tanh(0.4778*phq_8/all)
score_9 = 3*math.tanh(0.2442*phq_9/all)
score_0 = 3*math.tanh(23.3*phq_0/all)

score = 0.9*math.tanh(4.4*p)*(score_0+score_1+score_2+score_3+score_4+score_5+score_6+score_7+score_8+score_9)

print(score)